In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.models import resnet18
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import os

In [12]:
# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # standard ImageNet values
                         std=[0.229, 0.224, 0.225])
])

data_path = "/kaggle/input/openaimer-2025-track-2-training-data"

full_dataset = ImageFolder(root=data_path, transform=transform)

# Train/val split 
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

targets = [full_dataset.targets[i] for i in train_dataset.indices]

class_counts = Counter(targets)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}

sample_weights = [class_weights[label] for label in targets]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)  # Note: no shuffle
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

num_classes = len(full_dataset.classes)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = resnet18(pretrained=True)  
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 199MB/s]


In [5]:
def train(model, loader):
    model.train()
    for epoch in range(10):
        total_loss = 0
        loop = tqdm(loader, desc=f"Epoch [{epoch+1}/10]")
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        print(f">>> Epoch {epoch+1} finished. Total Loss: {total_loss:.4f}\n")
train(model, train_loader)

Epoch [1/10]: 100%|██████████| 1000/1000 [03:32<00:00,  4.70it/s, loss=0.307]


>>> Epoch 1 finished. Total Loss: 624.9473



Epoch [2/10]: 100%|██████████| 1000/1000 [03:15<00:00,  5.11it/s, loss=0.413]


>>> Epoch 2 finished. Total Loss: 414.5704



Epoch [3/10]: 100%|██████████| 1000/1000 [03:01<00:00,  5.52it/s, loss=0.242]


...

>>> Epoch 8 finished. Total Loss: 208.8075



Epoch [9/10]: 100%|██████████| 1000/1000 [02:28<00:00,  6.73it/s, loss=0.0522] 


>>> Epoch 9 finished. Total Loss: 177.9402



Epoch [10/10]: 100%|██████████| 1000/1000 [02:27<00:00,  6.76it/s, loss=0.00584]

>>> Epoch 10 finished. Total Loss: 164.3367



In [6]:
torch.save(model.state_dict(), "resnet18_teacher.pth")

In [13]:
def validate(model, val_loader):
    model.eval()  
    correct = 0
    total = 0
    with torch.no_grad():  
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")
    return accuracy

In [14]:
validate(model, val_loader)

Validation Accuracy: 78.78%


78.775

# Compression Processes

In [14]:
!pip install torch-pruning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [20]:
import torch
import torch.nn as nn
import torch_pruning as tp
from torchvision.models import resnet18

model = resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/kaggle/working/resnet18_teacher.pth"))
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

example_inputs = torch.randn(1, 3, 224, 224).to(device)

imp = tp.importance.MagnitudeImportance(p=2)  
pruner = tp.pruner.MagnitudePruner(
    model,
    example_inputs=example_inputs,
    importance=imp,
    iterative_steps=1,   
    pruning_ratio=0.3,   
    ignored_layers=[model.fc],  
)

# Prune!
pruner.step()

torch.save(model, "resnet18_pruned_actual.pth")

/tmp/ipykernel_31/2561685082.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/resnet18_teacher.pth"))


In [21]:
model = torch.load("/kaggle/working/resnet18_pruned_actual.pth")
model = model.to(device)

# Criterion and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Lower LR for fine-tuning

# Fine-tuning function
def finetune(model, loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct, total = 0, 0
        loop = tqdm(loader, desc=f"Fine-tune Epoch [{epoch+1}/{epochs}]")
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

        print(f">>> Epoch {epoch+1}: Loss = {total_loss:.4f}, Accuracy = {100. * correct / total:.2f}%\n")

finetune(model, train_loader, epochs=5)

torch.save(model.state_dict(), "resnet18_pruned_actual_finetuned.pth")

/tmp/ipykernel_31/473569755.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/kaggle/working/resnet18_pruned_actual.pth")
Fine-tune Epoch [1/5]: 100%|

>>> Epoch 1: Loss = 164.8191, Accuracy = 95.13%



Fine-tune Epoch [2/5]: 100%|██████████| 1000/1000 [02:09<00:00,  7.74it/s, acc=97.3, loss=0.111] 


>>> Epoch 2: Loss = 97.4809, Accuracy = 97.29%



Fine-tune Epoch [3/5]: 100%|██████████| 1000/1000 [02:08<00:00,  7.79it/s, acc=97.9, loss=0.175]  


>>> Epoch 3: Loss = 74.1831, Accuracy = 97.90%



Fine-tune Epoch [4/5]: 100%|██████████| 1000/1000 [02:07<00:00,  7.85it/s, acc=98.3, loss=0.00498]


>>> Epoch 4: Loss = 62.7956, Accuracy = 98.29%



Fine-tune Epoch [5/5]: 100%|██████████| 1000/1000 [02:09<00:00,  7.71it/s, acc=98.7, loss=0.0264] 


>>> Epoch 5: Loss = 47.2883, Accuracy = 98.72%



In [22]:
torch.save(model, "resnet18_pruned_actual_finetuned_model.pth")

In [4]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("/kaggle/input/compressedmodel/pytorch/default/1/model.pth")
model = model.to(device)
model.eval()  

def evaluate_model(model, val_loader, device):
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

accuracy = evaluate_model(model, val_loader, device)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

/tmp/ipykernel_31/3713926258.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/kaggle/input/compressedmodel/pytorch/default/1/model.pth")


Validation Accuracy: 93.73%


In [7]:
import torch
import torch.nn as nn
import time
from torchvision import models
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

original_model = models.resnet18(pretrained=True)  
original_model.fc = nn.Linear(original_model.fc.in_features, num_classes)
original_model = original_model.to(device)

state_dict = torch.load("/kaggle/input/basemodel/pytorch/default/1/resnet18_teacher.pth")

original_model.load_state_dict(state_dict, strict=False)  
original_model.eval()  

# Load pruned model
pruned_model = torch.load("/kaggle/input/compressedmodel/pytorch/default/1/model.pth")
pruned_model = pruned_model.to(device)
pruned_model.eval()

# Evaluation function
def evaluate_model(model, val_loader, device):
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            # Store predictions and true labels
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Measure inference time for original model
start_time = time.time()
original_accuracy = evaluate_model(original_model, val_loader, device)
original_inference_time = time.time() - start_time

# Measure inference time for pruned model
start_time = time.time()
pruned_accuracy = evaluate_model(pruned_model, val_loader, device)
pruned_inference_time = time.time() - start_time

# Model size comparison
original_model_size = sum(p.numel() for p in original_model.parameters()) * 4 / (1024 ** 2)  # Size in MB
pruned_model_size = sum(p.numel() for p in pruned_model.parameters()) * 4 / (1024 ** 2)  # Size in MB

# Print results
print(f"Original Model Accuracy: {original_accuracy * 100:.2f}%")
print(f"Pruned Model Accuracy: {pruned_accuracy * 100:.2f}%")
print(f"Original Model Size: {original_model_size:.2f} MB")
print(f"Pruned Model Size: {pruned_model_size:.2f} MB")
print(f"Original Model Inference Time: {original_inference_time:.4f} seconds")
print(f"Pruned Model Inference Time: {pruned_inference_time:.4f} seconds")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 203MB/s]
/tmp/ipykernel_31/2694876211.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is poss

Original Model Accuracy: 86.29%
Pruned Model Accuracy: 93.73%
Original Model Size: 42.83 MB
Pruned Model Size: 20.97 MB
Original Model Inference Time: 23.5036 seconds
Pruned Model Inference Time: 22.0461 seconds


# Save and Load model as state_dict

In [3]:
import torch

model = torch.load('/kaggle/input/compressedmodel/pytorch/default/1/model.pth')

/tmp/ipykernel_31/3711211622.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/kaggle/input/compressedmodel/pytorch/default/1/model.pth')


In [4]:
torch.save(model.state_dict(), 'pruned_model_state_dict.pth')

In [5]:
print(model)

ResNet(
  (conv1): Conv2d(3, 44, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return self.relu(out)


In [7]:
class PrunedResNet(nn.Module):
    def __init__(self, block=BasicBlock, num_classes=100):
        super().__init__()
        self.inplanes = 44

        self.conv1 = nn.Conv2d(3, 44, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(44)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 44, 2, stride=1)
        self.layer2 = self._make_layer(block, 89, 2, stride=2)
        self.layer3 = self._make_layer(block, 179, 2, stride=2)
        self.layer4 = self._make_layer(block, 358, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(358 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PrunedResNet()
model.load_state_dict(torch.load("/kaggle/working/pruned_model_state_dict.pth", map_location=device, weights_only=True))
model.to(device)
model.eval()

PrunedResNet(
  (conv1): Conv2d(3, 44, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tr

In [23]:
pruned_accuracy = evaluate_model(model, val_loader, device)

In [21]:
print(pruned_accuracy)

0.939
